# Data Visualization

This notebook demonstrates the process of visualizing data stored in Iceberg data lake using Dash. The following is a very simple example of how to visualize data from DiveDB. For a more comprehensive example, see the [data visualization](../dash/data_visualization.py) file.

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import importlib
import os
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duck_pond = DuckPond(os.environ["CONTAINER_ICEBERG_PATH"])

labels = duck_pond.conn.sql(f"""
    SELECT DISTINCT label
    FROM DataLake
    WHERE animal = 'oror-002'
    """).df()

labels = sorted([name for name in labels.label.to_list() if (name.startswith("signal") or name.startswith("logger"))])

app = Dash()

fig = go.Figure()
dff = duck_pond.get_data(    
    animal_ids="oror-002",
    frequency=10,
    labels=labels,
)

figures = {
    col: go.Figure()
    for col in dff.columns[1:3]
}

for idx, (col, fig) in enumerate(figures.items()):
    fig.add_trace(
        go.Scattergl(
            x=dff['datetime'],
            y=dff[col],
            mode='lines',
            name=col,
            yaxis=f'y{idx+1}'
        )
    )

app.layout = [
    # dcc.Dropdown(labels, labels[0], id='dropdown-selection'),    
    dcc.Graph(id=f'graph-content-{idx}', figure=fig) for idx, fig in enumerate(figures.values())
]

app.run(debug=True, host='0.0.0.0', port=8053)